In [11]:
import pydrake.all
import numpy as np
import sympy as sym

In [2]:
#define constants
t, g= sym.symbols('t g')
mc, mp = sym.symbols('m_c m_p')
L, fx = sym.symbols('L f_x ')

# define variables 
theta,z = sym.symbols(r'\theta, z')
z_d = sym.symbols('z_d')
theta_d = sym.symbols(r'\theta_d')

In [5]:
z_dd = 1/(mc + mp*sym.sin(theta)**2)*(fx + mp*sym.sin(theta)*(L*(theta_d)**2+g*sym.cos(theta)))
theta_dd = 1/(L*(mc + mp*sym.sin(theta)**2))*(-fx*sym.cos(theta)-mp*L*(theta_d)**2*sym.cos(theta)*sym.sin(theta)-(mc+mp)*g*sym.sin(theta))

x_vec = sym.Matrix([z,theta,z_d,theta_d])
vector_field = sym.Matrix([z_d, theta_d,z_dd, theta_dd])

In [6]:
vector_field

Matrix([
[                                                                                                                      z_d],
[                                                                                                                 \theta_d],
[                                       (f_x + m_p*(L*\theta_d**2 + g*cos(\theta))*sin(\theta))/(m_c + m_p*sin(\theta)**2)],
[(-L*\theta_d**2*m_p*sin(\theta)*cos(\theta) - f_x*cos(\theta) - g*(m_c + m_p)*sin(\theta))/(L*(m_c + m_p*sin(\theta)**2))]])

In [7]:
# Linearization
# calculate the Jacobian around equilibrim point
pfpx = vector_field.jacobian([z, theta, z_d, theta_d])
pfpu = vector_field.jacobian([fx])

In [9]:
pfpx


Matrix([
[0,                                                                                                                                                                                                                                                                                                                0, 1,                                                                  0],
[0,                                                                                                                                                                                                                                                                                                                0, 0,                                                                  1],
[0,                                                                                      -2*m_p*(f_x + m_p*(L*\theta_d**2 + g*cos(\theta))*sin(\theta))*sin(\theta)*cos(\theta)/(m_c + m_p*sin(\theta)**2)**2 + (-g*m_p*sin(\theta)

In [10]:
pfpu

Matrix([
[                                          0],
[                                          0],
[               1/(m_c + m_p*sin(\theta)**2)],
[-cos(\theta)/(L*(m_c + m_p*sin(\theta)**2))]])

In [17]:
# Convert a SymPy expression into a function
function_pfpx = sym.lambdify((g,fx,mc,mp,L,theta,theta_d,z,z_d), pfpx)
funciton_pfpu = sym.lambdify((g,fx,mc,mp,L,theta,theta_d,z,z_d), pfpu)
Ahat = function_pfpx(9.81,0,10,1,0.5,np.pi,0,0,0)
Bhat = funciton_pfpu(9.81,0,10,1,0.5,np.pi,0,0,0)

In [18]:
Ahat 

array([[ 0.   ,  0.   ,  1.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  1.   ],
       [ 0.   ,  0.981,  0.   ,  0.   ],
       [ 0.   , 21.582,  0.   , -0.   ]])

In [19]:
Bhat

array([[0. ],
       [0. ],
       [0.1],
       [0.2]])

In [24]:
import scipy.signal as sig
cartpole_gain = sig.place_poles(Ahat, Bhat,  [-1+2*1j, -1-2*1j, -3, -2]).gain_matrix

In [25]:
cartpole_gain

array([[-15.29051988, 220.55525994, -18.85830785,  44.42915392]])

In [40]:
from pydrake.all import LinearQuadraticRegulator, LinearSystem
Q = np.diag((10., 10., 1., 1.))
R = [1]
Chat = np.mat('0,0,0,0')
Dhat = [0]
cartpole_syst = LinearSystem(Ahat, Bhat, Chat, Dhat)
lqr = LinearQuadraticRegulator(cartpole_syst, Q, R)